In [1]:
# importovanje biblioteka koje ce nam biti potrebne kasnije
import numpy as np
import pandas as pd 
import time
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import model_selection
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

In [2]:
#Ucitavanje fajla sa podacima za trening
data = pd.read_csv("D:\\Vlada\\Milica master\\all_variables.csv")
X = data.drop('TARGET', axis = 1)
y = data['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
print('Dimenzije trening skupa: ', X_train.shape)
X_train.head()

Dimenzije trening skupa:  (246008, 1543)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,client_installments_AMT_PAYMENT_min_count,client_installments_AMT_PAYMENT_min_mean,client_installments_AMT_PAYMENT_min_max,client_installments_AMT_PAYMENT_min_min,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_PAYMENT_sum_count,client_installments_AMT_PAYMENT_sum_mean,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_PAYMENT_sum_min,client_installments_AMT_PAYMENT_sum_sum
181648,310536,Cash loans,F,N,N,2,90000.0,227520.0,13189.5,180000.0,...,57.0,3846.4800,10338.120,75.555,219249.360,57.0,160426.550,223200.000,32691.105,9144313.00
229245,365516,Cash loans,M,Y,Y,0,90000.0,161730.0,13095.0,135000.0,...,27.0,9602.7900,13984.650,3826.665,259275.330,27.0,84089.430,153913.500,22835.790,2270414.80
122525,242055,Cash loans,M,N,Y,0,135000.0,728847.0,26307.0,553500.0,...,4.0,12985.4700,12985.470,12985.470,51941.880,4.0,51958.484,51958.484,51958.484,207833.94
306311,454894,Cash loans,M,N,N,0,135000.0,474183.0,34636.5,391500.0,...,82.0,7269.0425,13692.825,4764.825,596061.500,82.0,759773.440,1170179.200,28595.521,62301424.00
300658,448321,Cash loans,F,N,Y,0,180000.0,254700.0,27558.0,225000.0,...,11.0,3102.6150,3102.615,3102.615,34128.766,11.0,37066.230,37066.230,37066.230,407728.53


In [ ]:
X_train.columns

In [ ]:
tmp=X_train[['NAME_CONTRACT_TYPE']]

In [ ]:
tmp

In [ ]:

# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

#funkcija koja deli na optimalne grupe
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(X_train,y_train)

In [ ]:
final_iv

In [ ]:
final_iv.to_csv("final_iv_new.csv",index = False, chunksize = 500)

In [ ]:
final_iv=pd.read_csv("C:\\Users\\Vlada\\Desktop\\Milica master\\final_iv.csv")

In [ ]:
final_iv[final_iv['VAR_NAME']=='EXT_SOURCE_3']

In [ ]:
#Varijable koje imaju informativnu vrednost iznad 0.1
IV.loc[IV['IV']>0.1]

In [ ]:
type(final_iv.loc[final_iv['VAR_NAME']=='EXT_SOURCE_1','WOE'])

In [ ]:
tmp = final_iv[final_iv['VAR_NAME']=='EXT_SOURCE_1'][['MIN_VALUE','WOE']][:-1] 
#poslednja grupa je NaN pa cemo je ukloniti

tmp['WOE'] = pd.to_numeric(tmp['WOE'])
tmp['MIN_VALUE'] = pd.to_numeric(tmp['MIN_VALUE'])

tmp.set_index('MIN_VALUE', inplace=True)

fig, ax = plt.subplots( figsize=(10, 5))

plt.title('WOE varijable EXT_SOURCE_1', fontsize=20)
plt.xlabel('Donja granica grupe', fontsize=11)
plt.ylabel('WoE', fontsize=11)

ax.plot(tmp)
ax.scatter(0.8, 0.152297)


ax.text(0.75, 0.22, "NaN")

plt.show()

In [ ]:
tmp = final_iv[final_iv['VAR_NAME']=='DAYS_BIRTH'][['MIN_VALUE','WOE']]

tmp['WOE'] = pd.to_numeric(tmp['WOE'])
tmp['MIN_VALUE'] = pd.to_numeric(tmp['MIN_VALUE'])

tmp.set_index('MIN_VALUE', inplace=True) 
fig, ax = plt.subplots( figsize=(10, 5))

plt.title('WOE varijable DAYS_BIRTH', fontsize=20)
plt.xlabel('Donja granica grupe', fontsize=11)
plt.ylabel('WoE', fontsize=11)

ax.plot(tmp)

plt.show()

In [ ]:
import time

#transformisemo pocetni uzorak tako sto umesto pocetne vrednosti varijable, stavljamo njenu WOE vrednost
def WOE_transformation(data,vars):
    start = time.time()

    for var in vars:

        tmp = final_iv[final_iv['VAR_NAME']==var][['MIN_VALUE','MAX_VALUE','WOE']]

        for idx, row in data.iterrows():

            for l,r,w in zip(tmp['MIN_VALUE'],tmp['MAX_VALUE'], tmp['WOE']):

                if (pd.isna(row[var]) or (row[var]>=l and row[var]<=r)):

                    data.loc[idx, var] = w

    end = time.time()
    print(round((end-start)/60,2))

In [ ]:
vars=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','bureau_DAYS_CREDIT_mean','client_installments_AMT_PAYMENT_min_sum']
WOE_transformation(X_train,vars)
WOE_transformation(X_test,vars)

In [ ]:
#WOE transformacija traje dugo pa cemo rezultate sacuvati u excelu
X_test_short_list=X_test[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','bureau_DAYS_CREDIT_mean','client_installments_AMT_PAYMENT_min_sum']]
X_train_short_list=X_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','bureau_DAYS_CREDIT_mean','client_installments_AMT_PAYMENT_min_sum']]

In [ ]:
X_train_short_list.to_csv('X_train_short_list.csv',index = False, chunksize = 500)

In [ ]:
X_test_short_list.to_csv('X_test_short_list.csv',index = False, chunksize = 500)

In [3]:
X_train_short_list=pd.read_csv('D:\\Vlada\\Milica master\\X_train_short_list.csv')
X_test_short_list=pd.read_csv('D:\\Vlada\\Milica master\\X_test_short_list.csv')

In [4]:
grid_model = LogisticRegression()
params = {'C': [10**i for i in range(-5, 5)]}
grid = model_selection.GridSearchCV(grid_model, param_grid=params, scoring='roc_auc', cv=5, return_train_score=True)

In [ ]:
grid.fit(X_train_short_list, y_train)

In [ ]:
grid.best_params_

In [ ]:
log_reg = LogisticRegression(class_weight = 'balanced',C = 100)
#Obucavamo model na trening setu
log_reg.fit(X_train_short_list, y_train)

In [ ]:
log_reg_pred = log_reg.predict_proba(X_test_short_list)
log_reg_pred

In [ ]:
print("Tacnost na trening setu: {:.3f}".format(log_reg.score(X_train_short_list, y_train)))
print("Tacnost na test setu: {:.3f}".format(log_reg.score(X_test_short_list, y_test)))

In [ ]:
predicted = log_reg.predict(X_test_short_list)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
logit_roc_auc = roc_auc_score(y_test, log_reg.predict(X_test_short_list))
fpr, tpr, thresholds = roc_curve(y_test, log_reg.predict_proba(X_test_short_list)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-specificnost')
plt.ylabel('osetljivost')
plt.title('ROC kriva')
plt.legend(loc="lower right")
plt.show()
plt.show()